In [1]:
import os
folder_path = '/Users/vvoo/Capstone_TS/data/api/'
folders = sorted([f for f in os.listdir(folder_path) if f.startswith('2024')])

for folder in folders:
    file_path = folder_path + folder
    print(file_path)

/Users/vvoo/Capstone_TS/data/api/2024-04-27
/Users/vvoo/Capstone_TS/data/api/2024-04-28
/Users/vvoo/Capstone_TS/data/api/2024-04-29
/Users/vvoo/Capstone_TS/data/api/2024-04-30
/Users/vvoo/Capstone_TS/data/api/2024-05-01
/Users/vvoo/Capstone_TS/data/api/2024-05-02
/Users/vvoo/Capstone_TS/data/api/2024-05-03
/Users/vvoo/Capstone_TS/data/api/2024-05-04
/Users/vvoo/Capstone_TS/data/api/2024-05-05
/Users/vvoo/Capstone_TS/data/api/2024-05-06
/Users/vvoo/Capstone_TS/data/api/2024-05-07
/Users/vvoo/Capstone_TS/data/api/2024-05-08
/Users/vvoo/Capstone_TS/data/api/2024-05-09
/Users/vvoo/Capstone_TS/data/api/2024-05-10
/Users/vvoo/Capstone_TS/data/api/2024-05-11
/Users/vvoo/Capstone_TS/data/api/2024-05-12
/Users/vvoo/Capstone_TS/data/api/2024-05-13
/Users/vvoo/Capstone_TS/data/api/2024-05-14
/Users/vvoo/Capstone_TS/data/api/2024-05-15
/Users/vvoo/Capstone_TS/data/api/2024-05-16
/Users/vvoo/Capstone_TS/data/api/2024-05-17
/Users/vvoo/Capstone_TS/data/api/2024-05-18
/Users/vvoo/Capstone_TS/data/api

In [3]:
import pandas as pd
import os
from tqdm import tqdm
import random

class concatDf:
    def __init__(self) -> None:
        # 폴더 내의 모든 csv 파일 목록 가져오기
        self.folder_path = '/Users/vvoo/Capstone_TS/data/api/'
        
        # 지하철 역 정보 불러오기
        self.up_station_csv = pd.read_csv('data/상행.csv', encoding='cp949')
        self.down_station_csv = pd.read_csv('data/하행.csv', encoding='cp949')
    
    def make_file(self):
        folders = sorted([f for f in os.listdir(self.folder_path) if f.startswith('2024')])
        print(folders[-15:])
        for folder in tqdm(folders[-15:]):
            file_path = self.folder_path + folder
            csv_files = [f for f in os.listdir(file_path) if f.startswith('train')]

            combined_csv = pd.concat([pd.read_csv(os.path.join(file_path, f)) for f in sorted(csv_files)], axis=0)
            combined_csv.to_csv(f'{file_path}/final_train.csv', index=True, encoding='utf-8-sig')
            
            # display(combined_csv)

            # display(self.make_time_series('7호선', '상행', '총신대입구(이수)', '남성', '1:20', combined_csv, folder))
            result_df = pd.DataFrame()

            # 상행에 대한 역정보
            for line, stat_name, prev_name, updn, time in tqdm(zip(self.up_station_csv['호선'], self.up_station_csv['역명'], self.up_station_csv['이전역'], self.up_station_csv['상하행'], self.up_station_csv['시간(분)'])):
                    tmp = self.make_time_series(line, updn, stat_name, prev_name, time, combined_csv, folder)
                    try:
                        result_df = pd.concat([result_df, tmp], ignore_index=False)
                    except:
                        result_df = tmp.copy()

            # 하행에 대한 역정보
            for line, stat_name, prev_name, updn, time in tqdm(zip(self.down_station_csv['호선'], self.down_station_csv['역명'], self.down_station_csv['이전역'], self.down_station_csv['상하행'], self.down_station_csv['시간(분)'])):
                    tmp = self.make_time_series(line, updn, stat_name, prev_name, time, combined_csv, folder)
                    try:
                        result_df = pd.concat([result_df, tmp], ignore_index=False)
                    except:
                        result_df = tmp.copy()

            # display(result_df)
            result_df.to_csv(f'{file_path}/combined_all.csv', index=True, encoding='utf-8-sig')

    def make_time_series(self, line, updn, stat_Nm, prev_Nm, time, combined_csv, folder):
        # api 생성 시간 기준
        combined_csv['recptnDt'] = pd.to_datetime(combined_csv['recptnDt'], format='%Y-%m-%d %H:%M:%S', errors='raise')
        combined_csv['time_zone'] = combined_csv['recptnDt'].dt.hour

        tmp = combined_csv[(combined_csv['statnNm'] == stat_Nm) & (combined_csv['arvlMsg3'] == prev_Nm)] # & (combined_csv['arvlMsg2'].isin(['전역 도착', '전역 출발', '전역 접근', '전역도착', '전역출발', '전역접근']))
        # display(tmp)

        
        def convert_to_seconds(time_str):
            minutes, seconds = map(int, time_str.split(':'))
            return minutes * 60 + seconds
        
        # 소요시간 이상치 처리
        time = convert_to_seconds(time)
        tmp['barvlDt'] = tmp['barvlDt'].apply(lambda x: random.randrange(time-30, time+30) if x > time-60 or x < time+60 else x) # 시간의 범위는 20초 이내
        # display(tmp)

        # if line == '2호선':
        #     if updn:
        #         tmp = combined_csv[(combined_csv['statnNm'] == stat_Nm) & (combined_csv['arvlMsg2'].isin(['전역 도착', '전역 출발', '전역 접근'])) & (combined_csv['arvlMsg3'] == prev_Nm)]
        #         updn = '내선'
        #     else:
        #         tmp = combined_csv[(combined_csv['statnNm'] == stat_Nm) & (combined_csv['arvlMsg2'].isin(['전역 도착', '전역 출발', '전역 접근'])) & (combined_csv['arvlMsg3'] == prev_Nm)]
        #         updn = '외선'
        # else:
        #     if updn:
        #         tmp = combined_csv[(combined_csv['statnNm'] == stat_Nm) & (combined_csv['arvlMsg2'].isin(['전역 도착', '전역 출발', '전역 접근'])) & (combined_csv['arvlMsg3'] == prev_Nm)]
        #         updn = '하행'
        #     else:
        #         tmp = combined_csv[(combined_csv['statnNm'] == stat_Nm) & (combined_csv['arvlMsg2'].isin(['전역 도착', '전역 출발', '전역 접근'])) & (combined_csv['arvlMsg3'] == prev_Nm)]
        #         updn = '상행'

        # Group by hour and calculate the average 'barvlDt'
        average_per_hour = tmp.groupby(tmp['time_zone'])['barvlDt'].mean().reset_index(name=f'{folder}')
        average_per_hour = average_per_hour.set_index('time_zone').transpose()

        average_per_hour['역명'] = stat_Nm
        average_per_hour['호선'] = line
        average_per_hour['상하행'] = updn
        average_per_hour['이전역'] = prev_Nm

        return average_per_hour
    

In [226]:
import warnings
warnings.filterwarnings("ignore")
TS = concatDf()
TS.make_file()

['2024-05-18', '2024-05-19', '2024-05-20', '2024-05-21', '2024-05-22', '2024-05-23', '2024-05-24', '2024-05-25', '2024-05-26', '2024-05-27', '2024-05-28', '2024-05-29', '2024-05-30', '2024-05-31', '2024-06-01']


300it [00:44,  6.79it/s]0:00<?, ?it/s]
300it [00:43,  6.91it/s]
300it [00:38,  7.88it/s]1:37<22:43, 97.38s/it]
300it [00:37,  8.00it/s]
300it [00:50,  5.96it/s]3:01<19:21, 89.31s/it]
300it [00:50,  5.93it/s]
300it [00:43,  6.84it/s]4:52<19:54, 99.54s/it]
300it [00:43,  6.96it/s]
300it [00:44,  6.75it/s]6:29<18:01, 98.31s/it]
300it [00:43,  6.88it/s]
300it [00:45,  6.57it/s]8:06<16:18, 97.88s/it]
300it [00:49,  6.06it/s]
300it [00:46,  6.46it/s]9:50<15:01, 100.14s/it]
300it [00:41,  7.16it/s]
300it [00:45,  6.63it/s]1:29<13:16, 99.55s/it] 
300it [00:44,  6.72it/s]
300it [00:47,  6.32it/s]3:09<11:38, 99.73s/it]
300it [00:47,  6.32it/s]
300it [00:48,  6.18it/s]4:53<10:06, 101.03s/it]
300it [00:52,  5.67it/s]
300it [00:46,  6.51it/s]16:45<08:41, 104.38s/it]
300it [00:47,  6.25it/s]
300it [00:42,  7.06it/s]18:30<06:59, 104.79s/it]
300it [00:47,  6.37it/s]
300it [00:42,  6.99it/s]20:09<05:08, 102.86s/it]
300it [00:49,  6.08it/s]
300it [00:44,  6.70it/s]21:49<03:24, 102.08s/it]
300it [00:42, 

- 역별 시계열 데이터 구성

In [4]:
import os
base_path = '/Users/vvoo/Capstone_TS/data/api/'
folders = sorted([f for f in os.listdir(folder_path) if f.startswith('2024')])

# file_path 내에 존재하는 combined_all.csv파일을 하나의 데이터프레임으로 합치기
# 각 날짜별 디렉토리에서 파일 읽기
all_data_frames = []
# 각 날짜별 디렉토리에서 파일 읽기
for date in folders:
    file_path = os.path.join(base_path, date, "combined_all.csv")
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, index_col=0)
        all_data_frames.append(df)
    else:
        print(f"File not found: {file_path}")

# 모든 데이터프레임을 하나로 병합
combined_df = pd.concat(all_data_frames, ignore_index=False)

# 병합된 데이터프레임 출력 또는 저장
combined_df = combined_df[['역명', '호선', '상하행', '이전역'] + [str(i) for i in range(5,24)]]
combined_df = combined_df[combined_df['이전역'] != '종점']
combined_df

File not found: /Users/vvoo/Capstone_TS/data/api/2024-05-17/combined_all.csv
File not found: /Users/vvoo/Capstone_TS/data/api/2024-06-02/combined_all.csv
File not found: /Users/vvoo/Capstone_TS/data/api/2024-06-03/combined_all.csv
File not found: /Users/vvoo/Capstone_TS/data/api/2024-06-04/combined_all.csv
File not found: /Users/vvoo/Capstone_TS/data/api/2024-06-05/combined_all.csv
File not found: /Users/vvoo/Capstone_TS/data/api/2024-06-06/combined_all.csv


,역명,호선,상하행,이전역,5,6,7,8,9,10,...,14,15,16,17,18,19,20,21,22,23
2024-04-27,도봉산,1,상행,도봉,NaN,NaN,NaN,NaN,NaN,NaN,...,76.25,95.714286,97.750000,76.000000,89.857143,103.142857,78.200000,89.400000,100.200000,89.000000
2024-04-27,도봉,1,상행,방학,NaN,NaN,NaN,NaN,NaN,NaN,...,86.00,81.500000,91.750000,86.400000,101.500000,90.666667,90.333333,91.833333,90.000000,104.500000
2024-04-27,방학,1,상행,창동,NaN,NaN,NaN,NaN,NaN,NaN,...,122.20,113.166667,114.200000,121.750000,119.125000,132.333333,123.200000,112.000000,136.400000,NaN
2024-04-27,창동,1,상행,녹천,NaN,NaN,NaN,NaN,NaN,NaN,...,109.50,120.833333,122.500000,121.500000,118.500000,125.833333,119.250000,110.166667,117.000000,123.000000
2024-04-27,녹천,1,상행,월계,NaN,NaN,NaN,NaN,NaN,NaN,...,118.50,124.625000,123.285714,131.333333,123.777778,115.428571,113.142857,118.500000,129.166667,149.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-01,"남한산성입구(성남법원,검찰청)",8,하행,산성,126.0,134.0,129.166667,125.400000,125.000000,118.166667,...,NaN,115.666667,140.125000,126.000000,133.428571,126.250000,126.000000,133.666667,134.428571,142.250000
2024-06-01,단대오거리,8,하행,"남한산성입구(성남법원,검찰청)",62.5,62.6,69.428571,71.500000,66.500000,60.666667,...,NaN,67.333333,62.400000,77.666667,73.857143,73.400000,72.666667,69.428571,74.333333,51.500000
2024-06-01,신흥,8,하행,단대오거리,69.5,61.2,59.000000,61.200000,66.500000,64.600000,...,NaN,75.666667,73.714286,73.750000,59.333333,77.750000,72.000000,78.800000,64.333333,98.000000
2024-06-01,수진,8,하행,신흥,79.0,74.2,62.400000,71.166667,64.600000,67.571429,...,NaN,67.250000,76.000000,67.000000,69.833333,71.250000,76.750000,67.833333,71.500000,62.500000


In [5]:
from tqdm import tqdm

# '역명'과 '이전역'을 기준으로 그룹화
grouped = combined_df.groupby(['역명', '이전역'])

# 각 그룹별로 별도의 데이터프레임 생성
grouped_dfs = {name: group for name, group in grouped}

# 각 그룹의 데이터프레임 확인
for name, group_df in tqdm(grouped_dfs.items()):
    # print(f"그룹: {name}")
    # print(group_df)
    print()

# # 예시 데이터프레임 저장 예시
for name, group_df in grouped_dfs.items():
    group_df.to_csv(f'/Users/vvoo/Capstone_TS/data/subway/group_{name}.csv', index=True)


100%|██████████| 577/577 [00:00<00:00, 719415.40it/s]


In [51]:
import pandas as pd
import os
base_path = '/Users/vvoo/Capstone_TS/data/subway/'
folders = sorted([f for f in os.listdir(base_path) if f.endswith('csv')])

# print(folders)
# 특정 날짜로 변경하는 함수
def shift_dates(data, days_offset):
    shifted_data = data.copy()
    shifted_data['date'] = pd.to_datetime(shifted_data['date']) - pd.Timedelta(days=days_offset)
    shifted_data['date'] = shifted_data['date'].apply(lambda x : x.strftime('%Y-%m-%d'))
    return shifted_data

def data_augmentation(base_path, file):
    # Load the existing data from your CSV file
    file_path = base_path + file
    df = pd.read_csv(file_path) # index_col=0

    # date 컬럼 생성
    df.rename(columns = {'Unnamed: 0' : 'date'}, inplace = True)

    # 데이터 범위 (2024-05-01 to 2024-05-31)로 데이터 필터링
    source_data = df[(df['date'] >= '2024-05-01') & (df['date'] <= '2024-05-31')]

    # 2024-03-20 ~ 2024-04-19 기간의 데이터를 생성 (2024-05-01 ~ 2024-05-31 데이터 이용)
    days_offset1 = (pd.Timestamp('2024-05-01') - pd.Timestamp('2024-03-20')).days
    augmented_data1 = shift_dates(source_data[source_data['date'] >= '2024-05-01'], days_offset1)
    # display(augmented_data1)

    # 2024-04-20 ~ 2024-04-30 기간의 데이터를 생성 (2024-05-04 ~ 2024-05-14 데이터 이용)
    days_offset2 = (pd.Timestamp('2024-05-04') - pd.Timestamp('2024-04-20')).days
    augmented_data2 = shift_dates(source_data[(source_data['date'] >= '2024-05-04')&(source_data['date'] <= '2024-05-14')], days_offset2)
    # display(augmented_data2)

    # 원본 데이터와 증강된 데이터를 결합
    combined_data = pd.concat([augmented_data1, augmented_data2, df[(df['date'] >= '2024-05-01')]]).reset_index(drop=True)
    # 'date' 열을 인덱스로 설정
    combined_data.set_index('date', inplace=True)

    # 인덱스 이름 제거
    combined_data.rename_axis(None, inplace=True)

    combined_data.to_csv(base_path+file, index=True)

    return

for folder in folders:
    data_augmentation(base_path, folder)

In [52]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from tbats import TBATS
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import numpy as np
import matplotlib.pyplot as plt
import random

def predTimeSeries(stat, prev, default):
    # 데이터 불러오기
    file_path = f"/Users/vvoo/Capstone_TS/data/subway/group_('{stat}', '{prev}').csv"
    root_df = pd.read_csv(file_path, index_col=0)

    # 데이터 확인
    df_index = root_df.index
    print(root_df.index)

    # 데이터 전처리
    df = pd.DataFrame()
    for i in df_index:
        tmp = pd.DataFrame(root_df.loc[[i]])
        tmp['Date'] = i
        tmp = tmp.melt(id_vars=['역명', '호선', '상하행', '이전역', 'Date'], var_name='Hour', value_name='Duration')
        
        # 'Hour'와 'Date'를 결합하여 'DateTime' 생성
        tmp['DateTime'] = tmp['Date'] + '_' + tmp['Hour']
        tmp['DateTime'] = pd.to_datetime(tmp['DateTime'], format='%Y-%m-%d_%H')
        
        df = pd.concat([df, tmp])

    # 'DateTime'을 인덱스로 설정
    df.set_index('DateTime', inplace=True)

    # 필요한 열만 선택 (예: 소요 시간)
    df = df[['Duration']]
    df['Duration'].fillna(random.randrange(default-10, default+10), inplace=True)

    return df
    # 예측값 출력
    # forecast_df.to_csv(f'/Users/vvoo/Capstone_TS/data/subway/pred/{stat}_{prev}.csv', index=True, encoding='utf-8-sig')

# 예제 실행
predTimeSeries('강남구청', '청담', 120)

Index(['2024-03-20', '2024-03-21', '2024-03-22', '2024-03-23', '2024-03-24',
       '2024-03-25', '2024-03-26', '2024-03-27', '2024-03-28', '2024-03-29',
       '2024-03-30', '2024-03-31', '2024-04-01', '2024-04-02', '2024-04-03',
       '2024-04-04', '2024-04-06', '2024-04-07', '2024-04-08', '2024-04-09',
       '2024-04-10', '2024-04-11', '2024-04-12', '2024-04-13', '2024-04-14',
       '2024-04-15', '2024-04-16', '2024-04-17', '2024-04-18', '2024-04-19',
       '2024-04-20', '2024-04-21', '2024-04-22', '2024-04-23', '2024-04-24',
       '2024-04-25', '2024-04-26', '2024-04-27', '2024-04-28', '2024-04-29',
       '2024-04-30', '2024-05-01', '2024-05-02', '2024-05-03', '2024-05-04',
       '2024-05-05', '2024-05-06', '2024-05-07', '2024-05-08', '2024-05-09',
       '2024-05-10', '2024-05-11', '2024-05-12', '2024-05-13', '2024-05-14',
       '2024-05-15', '2024-05-16', '2024-05-18', '2024-05-19', '2024-05-20',
       '2024-05-21', '2024-05-22', '2024-05-23', '2024-05-24', '2024-05-25',

,Duration
DateTime,
2024-03-20 05:00:00,92.000000
2024-03-20 06:00:00,92.375000
2024-03-20 07:00:00,90.230769
2024-03-20 08:00:00,84.619048
2024-03-20 09:00:00,92.733333
...,...
2024-06-01 19:00:00,96.571429
2024-06-01 20:00:00,94.750000
2024-06-01 21:00:00,91.222222


In [235]:
import math

# 지하철 역 정보 불러오기
up_station_csv = pd.read_csv('data/상행.csv', encoding='cp949')
down_station_csv = pd.read_csv('data/하행.csv', encoding='cp949')

# 역간 소요시간 예측
test_up = down_station_csv[['역명', '이전역', '시간(분)']]
for i in range(len(test_up)):
    input = test_up.iloc[i].values
    if input[1] == '종점':
        continue
    else:
        predTimeSeries(input[0], input[1], convert_to_seconds(input[2]))

Index(['2024-04-27', '2024-04-28', '2024-04-29', '2024-04-30', '2024-05-01',
       '2024-05-02', '2024-05-03', '2024-05-04', '2024-05-05', '2024-05-06',
       '2024-05-07', '2024-05-08', '2024-05-09', '2024-05-10', '2024-05-11',
       '2024-05-12', '2024-05-13', '2024-05-14', '2024-05-15', '2024-05-16',
       '2024-05-18', '2024-05-19', '2024-05-20', '2024-05-21', '2024-05-22',
       '2024-05-23', '2024-05-24', '2024-05-25', '2024-05-26', '2024-05-27',
       '2024-05-28', '2024-05-29', '2024-05-30', '2024-05-31', '2024-06-01'],
      dtype='object')
                     Duration
DateTime                     
2024-04-27 05:00:00      98.0
2024-04-27 06:00:00      98.0
2024-04-27 07:00:00      98.0
2024-04-27 08:00:00      98.0
2024-04-27 09:00:00      98.0
RUNNING THE L-BFGS-B CODE

           * * *



 This problem is unconstrained.


Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.70398D+00    |proj g|=  5.44328D-02

At iterate    5    f=  3.55382D+00    |proj g|=  8.85421D-02

At iterate   10    f=  3.50130D+00    |proj g|=  3.82100D-04

At iterate   15    f=  3.50123D+00    |proj g|=  1.08600D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     17     22      1     0     0   2.859D-05   3.501D+00
  F =   3.5012255764608091     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
Index(['2024-04-27', '2024-04-28', '2024-04-29', '202

 This problem is unconstrained.



At iterate    5    f=  3.42387D+00    |proj g|=  5.66252D-02

At iterate   10    f=  3.40832D+00    |proj g|=  1.25038D-02

At iterate   15    f=  3.40697D+00    |proj g|=  1.79810D-03

At iterate   20    f=  3.40696D+00    |proj g|=  1.08442D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     20     23      1     0     0   1.084D-05   3.407D+00
  F =   3.4069618302782434     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
Index(['2024-04-27', '2024-04-28', '2024-04-29', '2024-04-30', '2024-05-01',
       '2024-05-02', '2024-05-03', '2024-05-04', '2024-05-05', '2024-05-06',
       '2024-05-07', 

...


At iterate    5    f=  3.48459D+00    |proj g|=  1.08789D-01

At iterate   10    f=  3.47353D+00    |proj g|=  5.97830D-03

At iterate   15    f=  3.47300D+00    |proj g|=  1.45437D-04

At iterate   20    f=  3.47300D+00    |proj g|=  1.78290D-03

At iterate   25    f=  3.47288D+00    |proj g|=  4.82346D-03

At iterate   30    f=  3.47284D+00    |proj g|=  1.53817D-03

At iterate   35    f=  3.47283D+00    |proj g|=  1.65501D-03

At iterate   40    f=  3.47283D+00    |proj g|=  2.87592D-04

At iterate   45    f=  3.47282D+00    |proj g|=  4.80489D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     48     52   

 This problem is unconstrained.



At iterate    5    f=  3.53886D+00    |proj g|=  6.91836D-02

At iterate   10    f=  3.52021D+00    |proj g|=  4.76821D-04

At iterate   15    f=  3.52020D+00    |proj g|=  5.73066D-04

At iterate   20    f=  3.52020D+00    |proj g|=  4.68386D-04

At iterate   25    f=  3.52019D+00    |proj g|=  2.28168D-03

At iterate   30    f=  3.52018D+00    |proj g|=  3.27812D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     30     34      1     0     0   3.278D-05   3.520D+00
  F =   3.5201832657077654     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
Index(['2024-04-27', '2024-04-28', '2024-04-29', '2

 This problem is unconstrained.



At iterate    5    f=  2.43965D+00    |proj g|=  1.94381D-02

At iterate   10    f=  2.41714D+00    |proj g|=  4.62271D-03

At iterate   15    f=  2.41633D+00    |proj g|=  2.88376D-04

At iterate   20    f=  2.41629D+00    |proj g|=  4.84923D-03

At iterate   25    f=  2.41607D+00    |proj g|=  1.23309D-03

At iterate   30    f=  2.41605D+00    |proj g|=  3.25476D-04

At iterate   35    f=  2.41605D+00    |proj g|=  2.75990D-04

At iterate   40    f=  2.41605D+00    |proj g|=  1.00729D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     41     47      1     0     0   1.996D-05   2.416D+00
  F =   2.41604784671

In [ ]:
!pip install statsmodels

In [ ]:
!pip install tbats

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from tbats import TBATS
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import numpy as np
import matplotlib.pyplot as plt

# 데이터 불러오기 및 전처리
file_path = 'path_to_your_data.csv'
df = pd.read_csv(file_path, index_col=0, parse_dates=True)
df = df.asfreq('H')  # 시간 단위로 빈도 설정

# 데이터 분할 (학습 데이터와 테스트 데이터)
train_size = int(len(df) * 0.8)
train, test = df[:train_size], df[train_size:]

# SARIMA 모델 학습 및 예측
sarima_model = sm.tsa.SARIMAX(train['Duration'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 24))
sarima_fit = sarima_model.fit()
sarima_forecast = sarima_fit.forecast(steps=len(test))

# Holt-Winters 모델 학습 및 예측
holt_winters_model = ExponentialSmoothing(train['Duration'], seasonal='add', seasonal_periods=24).fit()
holt_winters_forecast = holt_winters_model.forecast(steps=len(test))

# T-BATS 모델 학습 및 예측
estimator = TBATS(seasonal_periods=[24])
t_bats_model = estimator.fit(train['Duration'])
t_bats_forecast = t_bats_model.forecast(steps=len(test))

# 평가 지표 계산
def calculate_metrics(test, forecast, model_name):
    mae = mean_absolute_error(test, forecast)
    mse = mean_squared_error(test, forecast)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(test, forecast)
    return pd.Series({'Model': model_name, 'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'MAPE': mape})

metrics_sarima = calculate_metrics(test['Duration'], sarima_forecast, 'SARIMA')
metrics_hw = calculate_metrics(test['Duration'], holt_winters_forecast, 'Holt-Winters')
metrics_tbats = calculate_metrics(test['Duration'], t_bats_forecast, 'T-BATS')

# 성능 지표 데이터프레임 생성
metrics_df = pd.DataFrame([metrics_sarima, metrics_hw, metrics_tbats])
print(metrics_df)


In [ ]:
test = pd.read_csv('/Users/vvoo/Capstone_TS/data/subway/pred/마포구청_망원.csv', index_col=0)
test